#Gen A.I Assignment
By: Minh Son Truong/shen.truong2017@gmail.com
##Setup Phase
import the important libraries from Huggingface

In [ ]:
!pip install transformers datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Define the model name and load the tokenizer and model
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Print out which device we're using (GPU or CPU)
print(device)

cuda


## Set up the method and test the model

In [ ]:
def recipe(text):
  inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
  recipe_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
  return tokenizer.decode(recipe_ids[0], skip_special_tokens=True)

In [ ]:
# Define a sample text for summarization
sample_text =     """
generate a dish with chicken, pasta, tomato sauce
"""

# Summarize the sample text using the pre-trained model (without fine-tuning)
pre_finetuned_recipe_generation = recipe(sample_text)
print("recipe generation before fine-tuning:", pre_finetuned_recipe_generation)

recipe generation before fine-tuning: chicken, pasta, tomato sauce


## Load dataset
- First import the library needed to load the dataset from hugging face


- Then, load the database that will be used

In [ ]:
from datasets import load_dataset

# Load the recipe dataset, which contains articles and summaries
dataset = load_dataset("Shengtao/recipe", split="train")

dataset_split = dataset.train_test_split(test_size=0.1)
eval_dataset = dataset_split['test']

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

recipe.csv:   0%|          | 0.00/64.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32722 [00:00<?, ? examples/s]

## Tokenize
- Set up a preprocess function to be able to train the model

- tokenize the necessary data from the database

In [ ]:
def preprocess_function(examples):
  # Extract the articles from the dataset
  inputs = [ingredient for ingredient in examples['ingredients']]

  # Tokenize the articles (inputs) with padding and truncation to a max length of 512
  model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True, return_tensors="pt")

  # Tokenize the summaries (labels) using the target tokenizer context
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['title'], max_length=128, padding="max_length", truncation=True, return_tensors="pt")

  # Attach the tokenized summaries as labels to the model inputs
  model_inputs["labels"] = labels["input_ids"]

  # Move the tokenized inputs and labels to the appropriate device (GPU/CPU)
  model_inputs = {k: v.to(device) for k, v in model_inputs.items()}

  return model_inputs

In [ ]:
# Tokenize the small training dataset
tokenized_train_dataset = dataset.map(preprocess_function, batched=True)

#tokenize the evaluation dataset for the finetuning process
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/32722 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3273 [00:00<?, ? examples/s]

## Set up training arguments
Give the training model specific configurations as I like

In [ ]:
from transformers import Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',              # Directory to save the model checkpoints
    eval_strategy="epoch",         # Evaluate the model at the end of every epoch
    learning_rate=2e-5,                  # Learning rate for the optimizer
    per_device_train_batch_size=8,       # Batch size for training
    per_device_eval_batch_size=8,        # Batch size for evaluation
    weight_decay=0.01,                   # Regularization to prevent overfitting
    save_total_limit=3,                  # Only keep the last 3 checkpoints
    num_train_epochs=2,                  # Set the number of repitition/epoch to 2 due to time limitation on google colab
    predict_with_generate=True,          # Enable text generation during evaluation
    logging_dir="./logs"                 # Directory for storing training logs
)

## Set up the trainer object
Using the Seq2SeqTrainer, we can train our model

In [ ]:
from transformers import Seq2SeqTrainer

# Create the trainer object
trainer = Seq2SeqTrainer(
    model=model,                         # The model to be trained
    args=training_args,                  # The training arguments defined earlier
    train_dataset=tokenized_train_dataset,  # The tokenized training dataset
    eval_dataset=tokenized_eval_dataset,  # The tokenized evaluation dataset
    tokenizer=tokenizer                  # The tokenizer to handle input and output
)

In [ ]:
# Train the model using the configuration
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.199400,0.172751
2,0.191700,0.167880


TrainOutput(global_step=8182, training_loss=0.684455811904538, metrics={'train_runtime': 3638.7628, 'train_samples_per_second': 17.985, 'train_steps_per_second': 2.249, 'total_flos': 1.2165418189848576e+16, 'train_loss': 0.684455811904538, 'epoch': 2.0})

## Evaluation
Evaluate the metrics belong to our model post train

In [ ]:
# Evaluate the model on the evaluation dataset
metrics = trainer.evaluate()

# Print the evaluation metrics
print(metrics)

{'eval_loss': 0.1678803265094757, 'eval_runtime': 50.3014, 'eval_samples_per_second': 65.068, 'eval_steps_per_second': 8.151, 'epoch': 2.0}


## Final test
- Set up the recipe method again to be called
- Run test

In [ ]:
def recipe(text):
  inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
  recipe_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
  return tokenizer.decode(recipe_ids[0], skip_special_tokens=True)

print(recipe("generate a dish with pasta, chicken, tomato sauce"))

Chicken and Tomatoes


In [ ]:
print(recipe("generate a dish with salmon and lime"))

Grilled Salmon


## Push the finetuned model to huggingface

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
trainer.push_to_hub("SMTruong/recipe_name_generator")

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/SMTruong/results/commit/88b7331a5cc9d0519861e8c03f74f739a03079d4', commit_message='SMTruong/recipe_name_generator', commit_description='', oid='88b7331a5cc9d0519861e8c03f74f739a03079d4', pr_url=None, pr_revision=None, pr_num=None)